In [1]:
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('ignore')

/opt/homebrew/anaconda3/envs/tf/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


### define training data

- Genism word2vec requires that a format of ‘list of lists’ be provided for training where every document contained in a list.
- Every list contains lists of tokens of that document.

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import seaborn as sns
import matplotlib.pyplot as plt

import re, os
import unicodedata
import json

import nltk
import nltk.sentiment
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from time import strftime

from wordcloud import WordCloud

from pprint import pprint

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

import acquire
import prepare_jag

import spacy
pd.set_option('display.max_colwidth', None)



In [2]:
# define training data
#Genism word2vec requires that a format of ‘list of lists’ be provided for training where every document contained in a list.
#Every list contains lists of tokens of that document.
corpus = [['dog','bites','man'], ["man", "bites" ,"dog"],["dog","eats","meat"],["man", "eats","food"]]

#Training the model
model_cbow = Word2Vec(corpus, min_count=1,sg=0) #using CBOW Architecture for trainnig
model_skipgram = Word2Vec(corpus, min_count=1,sg=1)#using skipGram Architecture for training 

In [4]:
# Import data the first time
df = prepare_jag.prep_train()

In [6]:
df.head(1)

,Unnamed: 0,id,case_num,pn_num,feature_num,feature_text,annotation,location,original,clean,stemmed,lemmatized,original_word_count
0,0,00016_000,0,16,0,Family-history-of-MI-OR-Family-history-of-myocardial-infarction,['dad with recent heart attcak'],['696 724'],"HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of ""heart beating/pounding out of my chest."" 2 days ago during a soccer game had an episode, but this time had chest pressure and felt as if he were going to pass out (did not lose conciousness). Of note patient endorses abusing adderall, primarily to study (1-3 times per week). Before recent soccer game, took adderrall night before and morning of game. Denies shortness of breath, diaphoresis, fevers, chills, headache, fatigue, changes in sleep, changes in vision/hearing, abdominal paun, changes in bowel or urinary habits. \r\nPMHx: none\r\nRx: uses friends adderrall\r\nFHx: mom with ""thyroid disease,"" dad with recent heart attcak\r\nAll: none\r\nImmunizations: up to date\r\nSHx: Freshmen in college. Endorses 3-4 drinks 3 nights / week (on weekends), denies tabacco, endorses trying marijuana. Sexually active with girlfriend x 1 year, uses condoms",hpi 17yo presents palpitations patient reports 3-4 months intermittent episodes heart beating/pounding chest 2 days ago soccer game episode time chest pressure felt going pass lose conciousness note patient endorses abusing adderall primarily study 1-3 times per week recent soccer game took adderrall night morning game denies shortness breath diaphoresis fevers chills headache fatigue changes sleep changes vision/hearing abdominal paun changes bowel urinary habits pmhx none rx uses friends adderrall fhx mom thyroid disease dad recent heart attcak none immunizations date shx freshmen college endorses 3-4 drinks 3 nights / week weekends denies tabacco endorses trying marijuana sexually active girlfriend x 1 year uses condoms,hpi 17yo present palpit patient report 3-4 month intermitt episod heart beating/pound chest 2 day ago dure soccer game episod thi time chest pressur felt go pass lose concious note patient endors abus adderal primarili studi 1-3 time per week befor recent soccer game took adderral night befor morn game deni short breath diaphoresi fever chill headach fatigu chang sleep chang vision/hear abdomin paun chang bowel urinari habit pmhx none rx use friend adderral fhx mom thyroid diseas dad recent heart attcak none immun date shx freshmen colleg endors 3-4 drink 3 night / week weekend deni tabacco endors tri marijuana sexual activ girlfriend x 1 year use condom,hpi 17yo present palpitation patient report 3-4 month intermittent episode heart beating/pounding chest 2 day ago soccer game episode time chest pressure felt going pas lose conciousness note patient endorses abusing adderall primarily study 1-3 time per week recent soccer game took adderrall night morning game denies shortness breath diaphoresis fever chill headache fatigue change sleep change vision/hearing abdominal paun change bowel urinary habit pmhx none rx us friend adderrall fhx mom thyroid disease dad recent heart attcak none immunization date shx freshman college endorses 3-4 drink 3 night / week weekend denies tabacco endorses trying marijuana sexually active girlfriend x 1 year us condom,142


In [7]:
#Make word lists by case_num_xs from clean words
case0_words = ' '.join(df[df.case_num == 0].clean).split()
case1_words = ' '.join(df[df.case_num == 1].clean).split()
case2_words = ' '.join(df[df.case_num == 2].clean).split()
case3_words = ' '.join(df[df.case_num == 3].clean).split()
case4_words = ' '.join(df[df.case_num == 4].clean).split()
case5_words = ' '.join(df[df.case_num == 5].clean).split()
case6_words = ' '.join(df[df.case_num == 6].clean).split()
case7_words = ' '.join(df[df.case_num == 7].clean).split()
case8_words = ' '.join(df[df.case_num == 8].clean).split()
case9_words = ' '.join(df[df.case_num == 9].clean).split()
all_words = ' '.join(df.clean).split()


In [8]:
all_words

['hpi',
 '17yo',
 'presents',
 'palpitations',
 'patient',
 'reports',
 '3-4',
 'months',
 'intermittent',
 'episodes',
 'heart',
 'beating/pounding',
 'chest',
 '2',
 'days',
 'ago',
 'soccer',
 'game',
 'episode',
 'time',
 'chest',
 'pressure',
 'felt',
 'going',
 'pass',
 'lose',
 'conciousness',
 'note',
 'patient',
 'endorses',
 'abusing',
 'adderall',
 'primarily',
 'study',
 '1-3',
 'times',
 'per',
 'week',
 'recent',
 'soccer',
 'game',
 'took',
 'adderrall',
 'night',
 'morning',
 'game',
 'denies',
 'shortness',
 'breath',
 'diaphoresis',
 'fevers',
 'chills',
 'headache',
 'fatigue',
 'changes',
 'sleep',
 'changes',
 'vision/hearing',
 'abdominal',
 'paun',
 'changes',
 'bowel',
 'urinary',
 'habits',
 'pmhx',
 'none',
 'rx',
 'uses',
 'friends',
 'adderrall',
 'fhx',
 'mom',
 'thyroid',
 'disease',
 'dad',
 'recent',
 'heart',
 'attcak',
 'none',
 'immunizations',
 'date',
 'shx',
 'freshmen',
 'college',
 'endorses',
 '3-4',
 'drinks',
 '3',
 'nights',
 '/',
 'week',
 '

In [9]:
corpus = all_words

In [10]:
#Training the model
model_cbow = Word2Vec(corpus, min_count=1,sg=0) #using CBOW Architecture for trainnig
model_skipgram = Word2Vec(corpus, min_count=1,sg=1)#using skipGram Architecture for training 

In [12]:
#Summarize the loaded model
print(model_cbow)


Word2Vec(vocab=39, vector_size=100, alpha=0.025)


TypeError: 'Word2Vec' object is not iterable

In [15]:

#Acess vector for one word
print(model_cbow['dog'])

TypeError: 'Word2Vec' object is not subscriptable

In [16]:
#Most similarity
model_cbow.most_similar('myocardial infarction')

AttributeError: 'Word2Vec' object has no attribute 'most_similar'

In [17]:
#Summarize the loaded model
print(model_skipgram)

#Summarize vocabulary
words = list(model_skipgram.wv.vocab)
print(words)


Word2Vec(vocab=39, vector_size=100, alpha=0.025)


AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

In [ ]:

#Acess vector for one word
print(model_skipgram['dog'])